use syntax tree to convert from and to any language


use a common library for language generation 

https://bitbucket.org/birkenfeld/pygments-main/src/default/pygments/lexers/haskell.py

https://github.com/PrismJS/prism/blob/master/components/prism-markdown.js

http://qbnz.com/highlighter/

https://stackoverflow.com/questions/171924/how-do-i-syntax-check-a-bash-script-without-running-it

bash -n scriptname

Improve on the babel-preset-php pattern of replacing code with a tree, just use templates and the tree creator on regular code blocks

https://gitlab.com/kornelski/babel-preset-php/blob/master/lib/trans.js

```
function translateAssignBlock(node, scope) {
    if (isVariableCreatingAssignment(node, scope)) {
        return makeVariableDeclaration(node.left.name, "var", translateExpr(node.right, scope), node.loc, scope);
    }
    return wrapInStatement(translateAssign(node, scope));
}
```

should be something like:
```
var ASSIGMENT_EXPRESSION = (leftName, right) => {
    var leftName = right;
}

function translateAssignBlock(node, scope) {
    if(//Assignment[/Identifier].includes(node)) {
        return getAST(ASSIGMENT_EXPRESSION, node)
    }
    return getAST(STATEMENT, node)
}
```

that way when they are written in multiple languages we can transpile any language, maybe

use this library as a reference for lots of languages https://github.com/seaneshbaugh/rosetta-euler

automatically translate https://projecteuler.net/archives to any language


TODO: use babel and a transpiling kernel from babel-preset-php, 

TODO: make a generic translator that does manipulations on queries based on templates






In [2]:
// sample definition for MatLab



In [ ]:
// definition for nodejs



## convert a tree to html?

Just so it can be parsed with XPath instead. Get them all working the same!

Crap modules I won't need any more:
- css
- ASTQ
- Acorn
- esprima
- escodegen
- esquery
- jsonpath
- jsonquery
- jspath
- json:select

TODO: get rid of jsonpath-plus, jsonstream, jspath, 



In [28]:
var importer = require('../Core');
var ASTQ  = require("astq");
var astq = new ASTQ();
var esquery = require('esquery');
var esprima = require('esprima'); 

function getArrayAST(selector, ctx, index) {
    if(typeof ctx === 'string') {
        ctx = esprima.parse(ctx, {range: true, tokens: true,
                                  comment: true, whitespace: true});
    }
    try {
        return astq.query(ctx, selector);
    } catch (e) {
        if(e.message.includes('query parsing failed')) {
            return esquery(ctx, selector);
        } else {
            throw e;
        }
    }
}

function BlockStatement({type, body}) {
    return `
<${type} type="${type}">
<body>
${body.map(statementToElement).join('\n')}
</body>
</${type}>
`
}

function FunctionDeclaration({type, id, params, body, async})  {
    return `
<${type} type="${type}" id="${id}">
<parameters>
${params.map(statementToElement).join('\n')}
</parameters>
<body>
${statementToElement(body)}
</body>
</${type}>`
}

// TODO: use mustache for this?
function Program ({type, body}) {
    return `
<${type} type="${type}">
<body>
${body.map(statementToElement).join('\n')}
</body>
</${type}>`;
}

function statementToElement(statement) {
    if(typeof global[statement.type] === 'function') {
        return global[statement.type](statement);
    }
}

function testASTArray(selector, search = '') {
    var cell = importer.interpret(search);
    var ast = getArrayAST(selector, `
function name(params) {
    
}
console.log()
`);
    /*
    [ Script {
    type: 'Program',
    body: [ [FunctionDeclaration], [ExpressionStatement] ],
    sourceType: 'script',
    range: [ 1, 45 ],
    comments: [],
    tokens:
    */
    
    /* expected output `
<program type="program">
    <function>
    </function>
    <expression>
    </expression>
</program>
` */
    console.log(ast[0].body[0])
    return statementToElement(ast[0]);
}

module.exports = testASTArray;

function htmlEntities(str) {
    return String(str).replace(/&/g, '&amp;').replace(/</g, '&lt;').replace(/>/g, '&gt;').replace(/"/g, '&quot;');
}

if(typeof $$ !== 'undefined') {
    var html = testASTArray('*', 'convert a tree to html')
    $$.html('<pre>' + htmlEntities(html) + '</pre>')
}


reading notebook /Users/briancullinan/jupyter_ops/Core/languages.ipynb
FunctionDeclaration {
  type: 'FunctionDeclaration',
  id:
   Identifier { type: 'Identifier', name: 'name', range: [ 10, 14 ] },
  params:
   [ Identifier { type: 'Identifier', name: 'params', range: [Array] } ],
  body:
   BlockStatement { type: 'BlockStatement', body: [], range: [ 23, 31 ] },
  generator: false,
  expression: false,
  async: false,
  range: [ 1, 31 ] }


ReferenceError: type is not defined